In [1]:
from pycontrol.experiment import Parameter, FloatParameter, IntParameter, Experiment
from pycontrol.stream import DataStream, DataAxis, DataStreamDescriptor, OutputConnector
from pycontrol.filters.io import WriteToHDF5

import itertools
import numpy as np
import asyncio
import time, sys
import h5py
import matplotlib.pyplot as plt

from pycontrol.filters.filter import Filter
from pycontrol.stream import InputConnector
from tqdm import tqdm, tqdm_notebook
import time

In [2]:
class ProgressBarExperiment(Experiment):

    # Parameters
    field          = FloatParameter(default=0.0, unit="T")
    measure_current= FloatParameter(default=3e-6, unit="A")
    voltage        = FloatParameter(default=0.0, unit="V")

    # Things coming back
    resistance = OutputConnector()

    def init_instruments(self):
        pass

    def init_streams(self):
        # Baked in data axes
        descrip = DataStreamDescriptor()
        self.resistance.set_descriptor(descrip)

    async def run(self):
        """This is run for each step in a sweep."""
        res = np.random.random(1)
        await self.resistance.push(res)
        logger.debug("Stream has filled {} of {} points".format(self.resistance.points_taken,
                                                            self.resistance.num_points() ))
        # Seemingly we need to give the filters some time to catch up here...
        await asyncio.sleep(0.5)

    def shutdown_instruments(self):
        print("Shutted down.")

In [3]:
class ProgressBar(Filter):
    """ Display progress bar(s) on the terminal.

    n: number of progress bars to be display, \
    corresponding to the number of axes (counting from outer most)
    """
    data = InputConnector()
    def __init__(self, num=1):
        super(ProgressBar,self).__init__()
        self.num    = num
        self.bars   = []
        self.w_id   = 0

    async def run(self):
        self.stream = self.data.input_streams[0]
        axes = self.stream.descriptor.axes
        num_axes = len(axes)
        totals = [self.stream.descriptor.num_points_through_axis(axis) for axis in range(num_axes)]
        chunk_sizes = [max(1,self.stream.descriptor.num_points_through_axis(axis+1)) for axis in range(num_axes)]
        self.num = min(self.num, num_axes)

        for i in range(self.num):
            self.bars.append(tqdm_notebook(total=totals[i]/chunk_sizes[i]))

        while True:
            if self.stream.done() and self.w_id==self.stream.num_points():
                break
            print("Prepare to get data...")
            new_data = np.array(await self.stream.queue.get()).flatten()
            print("Got some data.")
            while self.stream.queue.qsize() > 0:
                new_data = np.append(new_data, np.array(self.stream.queue.get_nowait()).flatten())
            self.w_id += new_data.size
            num_data = self.stream.points_taken
            for i in range(self.num):
                if num_data == 0:
                    # Reset the progress bar with a new one
                    self.bars[i].close()
                    self.bars[i] = tqdm_notebook(total=totals[i]/chunk_sizes[i])
                pos = int(10*num_data / chunk_sizes[i])/10.0 # One decimal is good enough
                if pos > self.bars[i].n:
                    self.bars[i].update(pos - self.bars[i].n)
                num_data = num_data % chunk_sizes[i]

In [4]:
exp = ProgressBarExperiment()
exp.sample = "Test ProgressBar"
exp.comment = "Test"
# wr = WriteToHDF5("test_data.h5")
progbar = ProgressBar(num=0)
# edges = [(exp.resistance, wr.data),(exp.resistance,progbar.data)]
edges = [(exp.resistance,progbar.data)]
exp.set_graph(edges)
exp.init_instruments()
main_sweep = exp.add_sweep(exp.field,np.linspace(0,-0.02,6))
exp.add_sweep(exp.measure_current,np.linspace(0,6,5))
exp.add_sweep(exp.voltage,np.linspace(1,5,3))

<SweptParameter: voltage>

In [5]:
exp.run_sweeps()
exp.shutdown_instruments()

Prepare to get data...
Got some data.
Prepare to get data...
Shutted down.


SystemExit: 0

To exit: use 'exit', 'quit', or Ctrl-D.


In [6]:
%tb

SystemExit: 0